#  <font color=red>Metagenome Pipeline</font>

This contains every line of code I use for my pipeline with explanations for some, it may be more efficient to copy and paste some of these lines into a script. This assumes you have already quality trimmed and assembled the metagenomes, and that you have 25 intein PSSMs generated from NR database (5 iterations).

## Searching a Metagenome with PSSM

This searches a metagenome with a PSSM (inteins used in this example but can be anything).
Requirements: <br />
assembled contigs = <font color=red> contigs.fasta </font> <br />
intein pssms = *.pssm  <br />
<br />
<br />

You have 25 intein PSSMs (amino acid), that have been generated from NR database; you will now use them to search a nucleotide database (assembled metagenome) for inteins.
    

In [ ]:
module load blast/2.7.1
for b in *.pssm; do tblastn -in_pssm "$b" -db contigs.fasta -out "$b".search -outfmt 6 -evalue 1e-10; done

PSSM searches like these will generate a ton of duplicate hits, since PSSMs have a fairly low specificty. The following will organize all your hits and get rid of duplicates.
mg_name.hits contains your final file with unique hits, sorted.

In [ ]:
cat *.search |cut -f 2 >all.hits
sort all.hits >sort.hits
uniq sort.hits >mg_name.hits

You will now use grep to get the sequences of the contigs that were identified as hits above. But first, grep only works on fasta files that are single line. You will remove \n to get rid of linebreaks in your assembled contigs file, then use grep to get the hit sequences.
<font color=blue>contigs.eol </font> will be the one line multifasta. <br />
At the end of the next block you will have <font color=green> *.seqfiles* </font> which are the sequence files of the hits from above.

In [ ]:
awk '!/^>/ { printf "%s", $0; n = "\n" } 
/^>/ { print n $0; n = "" }
END { printf "%s", n }
' contigs.fasta  >> contigs.eol


for filn in `cat mg_name.hits`; do grep -A 1 $filn contigs.eol> $filn.seqfile;done

Your <font color=green> *.seqfiles* </font> are all **possible** intein hits, but some of these are low quality, or bacteria like inteins, hedgehogs etc. You need to cull some of these hits via bitscore to make sure they can be reliably mapped later on. <br />

<br />
You now need to blast your <font color=green> *.seqfiles* </font> against a database (AA) of all recorded inteins to make sure they are the real thing. <br />

Requirements: <br />
inteindatabase (i can email) = intein.db

In [ ]:
for filn in `cat mg_name.hits`; do blastx -query $filn.seqfile -db intein.db -outfmt 6 -out $filn.blast ;done
for f in `cat all.hits`; do head -1 $f.blast >> all.tsbh; done

<font color=yellow>all.tsbh </font> contains only the top hit for each seqfile. You need to sort this file by bitscore and get rid of anything under 100-150. I highly recommend excel (data-> sort -> bitscore).  <br />
<br />
Now that your all.tsbh file only contains those hits with high bitscore, you can delete the seqfiles that didn't make the cut.
<br />
<br />

In [ ]:
cat all.tsbh |cut -f 1 > contigs.list #contigs.list is list of contigs that made it
mkdir genes #just a directory containing the good hits
for f in `cat contigs.list`; do mv $f.seqfile genes; done

You now have the good contigs, but some of these contigs will be extremly large. I usually have a maximum length of 10k bp sequence. To trim large contigs, look at <font color=purple>all.tsbh </font> , and make a separate file with only the large contigs (<font color=red>longcontigs.tsbh </font>). I usually do this in Excel, sort by query start or stop. <br /> Use the attached script: **contigtrimmer.pl**, which looks at the blast output of a large contig, gets about 1000 bases upstream and down stream of the intein and outputs that sequence.

In [ ]:
perl extein.pl longcontigs.tsbh

## Mapping

You now have appropriately sized <font color=green> *.seqfiles* </font> that contains inteins in them. You need to map metagenome reads back to them to determine the frequency of these "rare" genes. <br /> <br />
You need to prepare 2 sets of nucl sequences: <br />
1) <font color=green> *.seqfiles* </font> as they are , which represents extein/intein/extein <br />
2) <font color=green> *.seqfiles* </font> with the inteins removed, which represetns extein/extein <br />
<br />
Requirements: <br />
**extein.pl** which is attached <br />
<font color=green> *.seqfiles* </font> 
<font color=purple>all.tsbh </font> (you need to remake this file with the short contigs) <br />

<br />
To make the second set of sequences 2):

In [ ]:
perl extein.pl all.tsbh > extonly.txt
cat *.seqfile > fullseqs.txt

After you have both sets of sequences, you will use bowtie2 to map reads to these sequences (also called reference genomes).

In [ ]:
module load bowtie2/2.3.3.1
bowtie2-build extonly.txt extonly #builds bowtie indices
bowtie2-build fullseqs.txt fullseqs


bowtie2 -x extonly -U reads.fastq -S exmap.sam -p 10 #maps
bowtie2 -x fullseqs -U reads.fastq -S fullmap.sam -p 10 

samtools view -b -F 4 exmap.sam > exmap.bam #makes files much smaller, and convert to bam
samtools view -b -F 4 fullmap.sam > fullmap.bam

samtools sort exmap.bam -o exsort.bam #sorts based on position
samtools sort fullmap.bam -o fullsort.bam


After this you can now visualize your mapped reads, using Integrated Genome Viewer (or equivalent). For this you need to create an index file for the bam files and reference genomes.

In [ ]:
samtools faidx name.fasta 
samtools index -b name.bam

## Extein Background
You can then identify the gene/organism the intein is inserted to with an NR blast of the extein file. 

In [ ]:
blastx -query extonly.txt -db /isg/shared/databases/blast/nr -outfmt "6 qaccver salltitles qseq" -max_target_seqs 1 >> exteinbackground.tab

## The end?
At this point you should have mapped reads (.bam), list of all inteins in a metagenome (all.tsbh), and their host information. I do not have a great way to parse and catalogue all of this information at the moment, but collectively you can do phylogenetic analyses, create intein distribution tables, etc from all this.